In [1]:
import numpy as np
import pandas as pd

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute  import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest , chi2
from sklearn.tree import DecisionTreeClassifier

In [25]:
data=pd.read_csv(r'C:\Users\vasant\OneDrive\Desktop/titanic_train.csv',usecols=['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

In [27]:
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [33]:
#applying train_test_split on the data.
X_train,X_test,y_train,y_test= train_test_split(data.drop(columns=['Survived']),
                                                         data['Survived'],test_size=0.2,random_state=42)

In [37]:
#sample of X_train data
X_train.sample(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
57,3,male,28.5,0,0,7.2292,C
536,1,male,45.0,0,0,26.5500,S
800,2,male,34.0,0,0,13.0000,S
282,3,male,16.0,0,0,9.5000,S
332,1,male,38.0,0,1,153.4625,S


In [39]:
#sample of y_train data
y_train.sample(5)

762    1
864    0
487    0
742    1
114    0
Name: Survived, dtype: int64

In [50]:
#1 imputation transform
trf1= ColumnTransformer([
    ('impute_age', SimpleImputer(),[2]),
    ('impute_embarked', SimpleImputer(strategy="most_frequent"),[6])
], remainder='passthrough')

In [53]:
#one_hot_encoding
trf2= ColumnTransformer([
    ('sex_embarked_ohe', OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
], remainder='passthrough')

In [55]:
# scaling
trf3=ColumnTransformer([
    ('scale', MinMaxScaler(),slice(0,10))
    
])

In [63]:
#model selection
trf4=SelectKBest(score_func=chi2,k=5)

In [64]:
trf4

SelectKBest(k=5, score_func=<function chi2 at 0x00000281312173A0>)

In [65]:
# train the model using DecisionTreeClassifier
trf5=DecisionTreeClassifier()

In [68]:
pipeline=Pipeline([
    ('trf1',trf1),
    ('trf2', trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
    
])

In [76]:
pipeline

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('sex_embarked_ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x00000281312173A0>)),
                ('trf5', DecisionTreeClassifier())])

In [80]:
# alternative way to create pipeline(easiest way)
pipe=make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [79]:
pipe

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('sex_embarked_ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x00000281312173A0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [91]:
from sklearn import set_config
set_config(display='diagram')

In [92]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('sex_embarked_ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x00000281312173A0>)),
                ('trf5', DecisionTreeClassifier())])

In [96]:
#prediction
y_pred=pipeline.predict(X_test)

In [97]:
# count accuracy of the model
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

In [101]:
pipeline.named_steps['trf1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [102]:
pipe.score(X_test, y_test)

0.6256983240223464

In [1]:
pwd

'C:\\Users\\vasant'